리트코드 K 경유지 내 가장 저렴한 항공권: https://leetcode.com/problems/cheapest-flights-within-k-stops/

In [1]:
import collections
import heapq
import functools
import itertools
import re
import sys
import math
import bisect
from typing import *

In [2]:
case = {
    "n": 3,
    "flights": [[0,1,100],[1,2,100],[0,2,500]],
    "src": 0,
    "dst": 2,
    "k": 1
}

n은 노드의 수, flgits는 [출발, 도착, 비용] <br>
src 노드에서 출발하여 k개의 경유지를 거쳐 dst 노드로 가는 가장 비용이 적은 경로를 찾아야 한다.

# 다익스트라 알고리즘 응용 

In [3]:
def findCheapestPrice(n: int, flights: List[List[int]], src: int, dst: int, k: int) -> int:
    graph = collections.defaultdict(list)
    # 그래프 인접 리스트 구성
    for u, v, w in flights:
        graph[u].append((v, w))
        
    Q = [(0, src, k)] # 우선 순위 큐 [비용, 현재 출발 정점, 잔여 경유지 수]
    
    while Q:
        price, node, s = heapq.heappop(Q)
        if node == dst:
            return price
        if s >= 0: # 잔여 경유지 수가 남았으면
            for v, w in graph[node]:
                alt = price + w
                heapq.heappush(Q, (alt, v, s - 1))
    return -1

In [4]:
findCheapestPrice(**case)

200

다익스트라 알고리즘을 통해 탐색한다. <br>
경유지의 수도 고려되어야 하므로 우선순위 큐에 넣을 때 잔여 경유지 수도 넣어주고 경유할 때 하나씩 빼주며 만약 정해진 k보다 많은 경유지를 거쳐야 하는 경로는 탐색되지 않게 조건문을 설정한다.

## Time Limeted Exceeded 

위의 풀이를 리트코드에 제출하면 시간 제한에 걸린다. 책을 보고 작성한 거라 내가 뭔가 잘못 썼나 싶어서 책 깃헙에 가서 복사해서 붙여 넣었는데도 TLE가 발생한다.

In [5]:
tle_case = {
    "n": 13,
    "flights": [[11,12,74],[1,8,91],[4,6,13],[7,6,39],[5,12,8],[0,12,54],[8,4,32],[0,11,4],[4,0,91],[11,7,64],[6,3,88],[8,5,80],[11,10,91],[10,0,60],[8,7,92],[12,6,78],[6,2,8],[4,3,54],[3,11,76],[3,12,23],[11,6,79],[6,12,36],[2,11,100],[2,5,49],[7,0,17],[5,8,95],[3,9,98],[8,10,61],[2,12,38],[5,7,58],[9,4,37],[8,6,79],[9,0,1],[2,3,12],[7,10,7],[12,10,52],[7,2,68],[12,2,100],[6,9,53],[7,4,90],[0,5,43],[11,2,52],[11,8,50],[12,4,38],[7,9,94],[2,7,38],[3,7,88],[9,12,20],[12,0,26],[10,5,38],[12,8,50],[0,2,77],[11,0,13],[9,10,76],[2,6,67],[5,6,34],[9,7,62],[5,3,67]],
    "src": 10,
    "dst": 1,
    "k": 10
}

In [6]:
#findCheapestPrice(**tle_case)

에러가 나타나는 테스트 케이스는 위와 같다. 상당한 시간이 소요된다. 무언가 불필요한 연산이 계속되는 것 같다. <br>
이 코드에는 이전에 방문한 노드를 체크하는 알고리즘이 없다. 그래서 노드가 많고 최단 경로가 없는 테스트 케이스에서 같은 경로를 반복하는 순환에 빠져 TLE가 발생하는 것 같다.

In [7]:
def findCheapestPrice(n: int, flights: List[List[int]], src: int, dst: int, k: int) -> int:
    graph = collections.defaultdict(list)
    # 그래프 인접 리스트 구성
    for u, v, w in flights:
        graph[u].append((v, w))
        
    Q = [(0, src, k)] # 우선 순위 큐 [비용, 현재 출발 정점, 잔여 경유지 수]
    visited = {}
    
    while Q:
        price, node, s = heapq.heappop(Q)
        if node == dst:
            return price
        
        if node not in visited or visited[node] < s:
            visited[node] = s
            if s >= 0: # 잔여 경유지 수가 남았으면
                for v, w in graph[node]:
                    alt = price + w
                    heapq.heappush(Q, (alt, v, s - 1))
    return -1

In [8]:
findCheapestPrice(**tle_case)

-1

방문한 노드와 단계를 저장할 visted 딕셔너리를 만들어준다. <br>
단계는 잔여 경유지 수를 통해 세 줄 것이다. 경로를 일종의 층이 있는 트리라고 생각하고 층을 잔여 경유지 수로 세주는 것이다. <br>
잔여 경유지 수가 x일 때 해당 노드를 방문했다고 기록해놓고 이후에 그 노드를 heappop해서 방문하려고 할 때 현재 잔여 경유지 수 s보다 x가 작거나 같다면 이전 단계에서 방문한 적 있는 것이므로 재방문하지 않는다. <br>
s보다 x가 크다면 이후 단계에서 방문하는 노드이기 때문에 재방문해도 상관없다.